# Save pcigale fit results in a loop


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNRS
- creation date : 2024-02-08
- uddate : 2024-02-08

Find pcigale here https://cigale.lam.fr



- adaptation : Sylvie Dagoret-Campagne from  https://cigale.lam.fr


In [ ]:
#%pylab widget
#import matplotlib.pyplot as plt
#matplotlib.rcParams['figure.figsize'] = [9.,6.]

In [ ]:
#%pylab widget
import matplotlib.pyplot as plt
%matplotlib inline

import matplotlib.colors as colors
import matplotlib.cm as cmx
from matplotlib.colors import ListedColormap

import matplotlib.gridspec as gridspec

plt.rcParams['figure.figsize'] = [9.,6.]
import seaborn as sns

In [ ]:
#bwr_map = plt.get_cmap('bwr')
#reversed_map = bwr_map.reversed() 
#cNorm = colors.Normalize(0., vmax=1.)
#scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=bwr_map)
#all_colors = scalarMap.to_rgba(df["zobs"].values, alpha=1)


In [ ]:
import numpy as np

from IPython.display import display
from ipywidgets import widgets
from matplotlib import pyplot as plt

from astropy.table import Table
from astropy.io import fits
from astropy import units as u
from astropy import constants as ctes 
from scipy.constants import c

from pcigale import sed
from pcigale import sed_modules as modules
from pcigale.warehouse import SedWarehouse
from pcigale.data import SimpleDatabase
import os,re

In [ ]:
c

In [ ]:
path = "20240205_181701_out"

In [ ]:
def get_fitresults_dict(the_path):
    """
    paramters:
     - path : path where pcigale write results
    """
    all_files_runpcigale = sorted(os.listdir(the_path))
    spec_dict = {}
    for idx,filename in enumerate(all_files_runpcigale):

        # skip all file that are not ended by fits
        if not re.search('.*[.]fits$',filename):
            continue

        # 1) decode the spec number
        str_spec_found = re.findall('(^SPEC.+?)_.*', filename)
        #print("\t", idx," ",filename," ", str_spec_found)
        if len( str_spec_found) == 0:
            print(f"Skip filename {filename}")
            continue
        str_spec_found = str_spec_found[0]
  

        if str_spec_found in spec_dict.keys():
            #print("Found in dict ", spec_dict[str_spec_found])
            pass
        else:
            #print(f" ! {str_spec_found} NOT found in dict ")
            spec_dict[str_spec_found] = {"best_model": None,"SFH":None}

        d = spec_dict[str_spec_found]
    
        # 2) decode thefilename
        str_file_found = re.findall('^SPEC.+_(.*)[.]fits$', filename)[0]
        #print(idx,filename,str_file_found)

        if str_file_found == "SFH":
            d["SFH"] = filename
        elif str_file_found == "model":
            d["best_model"] = filename

        spec_dict[str_spec_found] = d
        
    return spec_dict
               
    

## Input

### Dictionary of file

In [ ]:
spec_dict = get_fitresults_dict(path)

### Select spectrum

In [ ]:
index = 10
spec_name = list(spec_dict.keys())[index]
spec_name

In [ ]:
spec_dict[spec_name]

In [ ]:
file_best_model = os.path.join(path,spec_dict[spec_name]['best_model'])
file_sfh = os.path.join(path,spec_dict[spec_name]['SFH'])

In [ ]:
t_best = Table.read(file_best_model)
t_sfh = Table.read(file_sfh)

In [ ]:
t_best[:50]

In [ ]:
list(t_best.columns)

## Read all results

In [ ]:
t_res = Table.read(os.path.join(path,'results.fits'))

In [ ]:
t_res[:4]

### Selection 

In [ ]:
cut_select = t_res["id"] == spec_name

In [ ]:
t_res_sel = t_res[cut_select]
t_res_sel

In [ ]:
t_res_sel["best.reduced_chi_square"][0]

### Read data

In [ ]:
file_data = os.path.join(path,"observations.fits")
FORS2= Table.read(file_data)
FORS2.add_index('id')

In [ ]:
FORS2[:4]

### Select

In [ ]:
cut_fors2select = FORS2["id"] == spec_name

In [ ]:
FORS2_sel = FORS2[cut_fors2select]

In [ ]:
FORS2_sel["galex.FUV"]

### Filters

In [ ]:
BANDS = [
    col for col in FORS2.colnames if not col.endswith("_err") 
    and not col.startswith("line") 
    and not col in ["id", "redshift", "distance"]
]

#with Database() as d:
#    BANDS_WAVE = np.array([d.get_filter(name).pivot_wavelength for name in BANDS])


with SimpleDatabase('filters') as d:
    #BANDS_WAVE = np.array([d.get(name).pivot_wavelength for name in BANDS])
    BANDS_WAVE = np.array([d.get(name=name).pivot for name in BANDS])
    

In [ ]:
BANDS_best = [ "best." + bandname for bandname in BANDS]

In [ ]:
band_sed_fluxes = np.array([ t_res_sel[bandname][0] for bandname in  BANDS_best])

# From pcigale

In [ ]:
with SimpleDatabase('filters') as d:
    #BANDS_WAVE = np.array([d.get(name).pivot_wavelength for name in BANDS])
    BANDS_WAVE = np.array([d.get(name=name).pivot for name in BANDS])
    BANDS_NAME = BANDS
dict_filters = dict(zip(BANDS_NAME, BANDS_WAVE)) 

In [ ]:
dict_filters 

In [ ]:
#obs= FORS2_sel.to_pandas().to_dict()
obs = FORS2_sel
obs

In [ ]:
series = ["model" ,"stellar_attenuated","stellar_unattenuated","nebular","dust"]

In [ ]:
def plots_frompcigale(t_best_model, mod,obs, filters , series=series,sed_type = "mJy", xrange = [10.*1e-3,5e4*1e-3],yrange = [False,False]):
    """

    parameters :
       t_best_model : Table of best fitted model (wrt wavelength)
       mod : Table of all fit result (line selected for the current spectrum)
       obs : Table of observed flux (in mJy)
       filters: dictionnary of filtername and pivot wavelength
       series : what to plot
       sed_type : mJy  Fnu in obs frame, lum : Llambda in restframe
       xrange : wavelength ranage in microns xrange = [100.*1e-3,1e4*1e-3]
    
    Need to convert flux density in mJy (mW/m2/Hz) into Luminosity W/nm and vice versa

    Fnu = 1/(4piDL^2) * lambda^2/c * L_lambda
    L_lambda = (4piDL^2) * (c/lambda^2) * F_nu

    No K correction is calculated
    
    """

    np.seterr(invalid="ignore")

    if True:
        # get the table containning all seds (wrt wavelength)
        sed = t_best_model
        # the best model
        # wavelengths from nm to microns 
        wavelength_spec = sed["wavelength"].data * 1e-3
        
        # pivot wavelengths in filter from nm to microns
        filters_wl = (np.array([filt_wl for filt_wl in filters.values()]) * 1e-3)

        
        # retrieve observed fluxes (in mJy)
        obs_fluxes = np.array([obs[filt].data[0] for filt in filters.keys()])
        obs_fluxes_err = np.array([obs[filt + "_err"].data[0] for filt in filters.keys()])
        
        # retrieve the model fluxes (in mJy)
        mod_fluxes = np.array(
            [
                mod["best." + filt].data[0]
                if "best." + filt in mod.colnames
                else np.nan
                for filt in filters.keys()
            ])

        # retrieve the redshift from the observation
        if obs["redshift"].data[0] >= 0:
            z = float(obs["redshift"].data[0])
        else:  # Redshift mode
            z = float(mod["best.universe.redshift"].data[0])

        # the redshift factor   
        zp1 = 1.0 + z

        # surface of the sphere centered on the object of radius distance-luminosity
        surf = 4.0 * np.pi * mod["best.universe.luminosity_distance"].data[0] ** 2

        # define the range if wavelength (in microns)
        xmin = 0.9 * np.min(filters_wl) if xrange[0] is False else xrange[0]
        xmax = 1.1 * np.max(filters_wl) if xrange[1] is False else xrange[1]

        if sed_type == "lum":
            # L_lambda = (4piDL^2) * (c/lambda^2) * F_nu
            # 1 mJy = 10^-29 W/m2/Hz
            # Every values converted to SI 
            # Convert wavelength from microns to meters
            # need to convert lumi from W/m to W/nm 
            k_corr_SED = (1e-29 * surf * ctes.c.value/ (filters_wl * 1e-6)**2)*1e-9

            #print("k_corr_SED --> W/m2 = ",k_corr_SED)
            
            # convert fluxes from mJy to W/nm
            obs_fluxes *= k_corr_SED
            obs_fluxes_err *= k_corr_SED
            mod_fluxes *= k_corr_SED

            #for cname in sed.colnames[1:]:

            # I think this should not be applied (why 1e3)
            for cname in sed.colnames[2:]:
                #sed[cname] *= wavelength_spec * 1e3
                sed[cname] *=  1e0
                
            # set source rest frame
            filters_wl /= zp1
            wavelength_spec /= zp1
            xmin /= zp1
            xmax /= zp1
            
        elif sed_type == "mJy":
            k_corr_SED = 1.0
            # 1 W/m2/Hz = 10^29 mJy
            #Fnu = 1/(4piDL^2) * lambda^2/c * L_lambda
            fact = 1e29 * 1e-3 * wavelength_spec ** 2 / ctes.c.value / surf

            #print("fact --> mJy = ",fact)
            
            #for cname in sed.colnames[1:]:
            for cname in sed.colnames[2:]:
                sed[cname] *= fact
        else:
            print("ERROR Unknown plot type.")

        wsed = np.where((wavelength_spec > xmin) & (wavelength_spec < xmax))
       

        
        print("================ START the figure ================================================================")

        #series = ["model" ,"stellar_attenuated","stellar_unattenuated","nebular","dust"]

        #'wavelength',
        # 'Fnu',
        # 'L_lambda_total',
        # 'stellar.old',
        # 'stellar.young',
        # 'nebular.absorption_old',
        # 'nebular.absorption_young',
        # 'nebular.lines_old',
        # 'nebular.lines_young',
        # 'nebular.continuum_old',
        # 'nebular.continuum_young',
        # 'attenuation.stellar.old',
        # 'attenuation.stellar.young',
        # 'attenuation.nebular.lines_old',
        # 'attenuation.nebular.lines_young',
        # 'attenuation.nebular.continuum_old',
        # 'attenuation.nebular.continuum_young',
        # 'dust',
        # 'igm'


        figure = plt.figure()
        gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1])
        
        #if (sed.columns[1][wsed] > 0.0).any():
        if (sed.columns[0][wsed].data > 0.0).any():    
            
            ax1 = plt.subplot(gs[0])
            ax2 = plt.subplot(gs[1])

            # Stellar emission
            if ( "stellar_attenuated" in series and "stellar.young" in sed.columns):
                spectrum =  sed["stellar.young"].data[wsed]  + sed["stellar.old"].data[wsed]

                if "nebular.absorption_young" in sed.columns:
                    spectrum += sed["nebular.absorption_young"].data[wsed]
                    spectrum += sed["nebular.absorption_old"].data[wsed]

                if "attenuation.stellar.young" in sed.columns:
                    spectrum += sed["attenuation.stellar.young"].data[wsed]
                    spectrum += sed["attenuation.stellar.old"].data[wsed]

                ax1.loglog(
                        wavelength_spec[wsed],
                        spectrum,
                        label="Stellar attenuated",
                        color="gold",
                        marker=None,
                        nonpositive="clip",
                        linestyle="-",
                        linewidth=1.0,
                    )
                #print("stellar_attenuated spectrum \t",spectrum)

            if ("stellar_unattenuated" in series and "stellar.young" in sed.columns):

                spectrum = sed["stellar.old"].data[wsed] + sed["stellar.young"].data[wsed]
                
                ax1.loglog(
                        wavelength_spec[wsed],
                        spectrum,
                        label="Stellar unattenuated",
                        color="xkcd:deep sky blue",
                        marker=None,
                        nonpositive="clip",
                        linestyle="--",
                        linewidth=1.0,
                    )
                #print("stellar_unattenuated spectrum \t",spectrum)

            # Nebular emission
            if "nebular" in series and "nebular.lines_young" in sed.columns:
                spectrum = (
                        sed["nebular.lines_young"].data[wsed]
                        + sed["nebular.lines_old"].data[wsed]
                        + sed["nebular.continuum_young"].data[wsed]
                        + sed["nebular.continuum_old"].data[wsed]
                    )

                if "attenuation.nebular.lines_young" in sed.columns:
                        spectrum += sed["attenuation.nebular.lines_young"].data[wsed]
                        spectrum += sed["attenuation.nebular.lines_old"].data[wsed]
                        spectrum += sed["attenuation.nebular.continuum_young"].data[wsed]
                        spectrum += sed["attenuation.nebular.continuum_old"].data[wsed]

                ax1.loglog(
                        wavelength_spec[wsed],
                        spectrum,
                        label="Nebular emission",
                        color="xkcd:true green",
                        marker=None,
                        nonpositive="clip",
                        linewidth=1.0,
                    )
                #print("nebular  spectrum \t",spectrum)

            # Dust emission Draine & Li
            if "dust" in series and "dust.Umin_Umin" in sed.columns:
                spectrum =  sed["dust.Umin_Umin"].data[wsed] + sed["dust.Umin_Umax"].data[wsed]
                ax1.loglog(
                        wavelength_spec[wsed],spectrum,
                        label="Dust emission",
                        color="xkcd:bright red",
                        marker=None,
                        nonpositive="clip",
                        linestyle="-",
                        linewidth=1.0,
                    )
                #print("Dust Draine Li  spectrum \t",spectrum)

            # Dust emission Dale
            if "dust" in series and "dust" in sed.columns:
                spectrum = sed["dust"].data[wsed]
                ax1.loglog(
                        wavelength_spec[wsed],
                        spectrum,
                        label="Dust emission",
                        color="xkcd:bright red",
                        marker=None,
                        nonpositive="clip",
                        linestyle="-",
                        linewidth=1.0,
                    )
                #print("Dust spectrum \t",spectrum)

            # AGN emission
            if "agn" in series and ( "agn.fritz2006_torus" in sed.columns or "agn.SKIRTOR2016_torus" in sed.columns):
                if "agn.fritz2006_torus" in sed.columns:
                    agn_sed = (
                            sed["agn.fritz2006_polar_dust"]
                            + sed["agn.fritz2006_torus"]
                            + sed["agn.fritz2006_disk"]
                        )
                elif "agn.SKIRTOR2016_torus" in sed.columns:
                    agn_sed = (
                            sed["agn.SKIRTOR2016_polar_dust"]
                            + sed["agn.SKIRTOR2016_torus"]
                            + sed["agn.SKIRTOR2016_disk"]
                        )
                if "xray.agn" in sed.columns:
                    agn_sed += sed["xray.agn"]
                if "radio.agn" in sed.columns:
                    agn_sed += sed["radio.agn"]
                    
                ax1.loglog(
                        wavelength_spec[wsed],
                        agn_sed[wsed],
                        label="AGN emission",
                        color="xkcd:apricot",
                        marker=None,
                        nonpositive="clip",
                        linestyle="-",
                        linewidth=1.0,
                    )

            # Radio emission
            if "radio" in series and "radio.sf_nonthermal" in sed.columns:
                ax1.loglog(
                        wavelength_spec[wsed],
                        sed["radio.sf_nonthermal"][wsed],
                        label="Radio SF nonthermal",
                        color="brown",
                        marker=None,
                        nonpositive="clip",
                        linestyle="-",
                        linewidth=1.0,
                    )

            if "model" in series:
                ax1.loglog(
                        wavelength_spec[wsed],
                        sed["L_lambda_total"].data[wsed],
                        label="Model spectrum",
                        color="k",
                        nonpositive="clip",
                        linestyle="-",
                        linewidth=1.5,
                    )
                #print("model", sed["L_lambda_total"].data[wsed])

            
            ax1.set_autoscale_on(False)
            
            s = np.argsort(filters_wl)
            filters_wl = filters_wl[s]
            mod_fluxes = mod_fluxes[s]
            obs_fluxes = obs_fluxes[s]
            obs_fluxes_err = obs_fluxes_err[s]
            ax1.scatter(
                    filters_wl,
                    mod_fluxes,
                    marker="o",
                    color="xkcd:strawberry",
                    s=8,
                    zorder=3,
                    label="Model fluxes",
                )
            mask_ok = np.logical_and(obs_fluxes > 0.0, obs_fluxes_err > 0.0)
            ax1.errorbar(
                    filters_wl[mask_ok],
                    obs_fluxes[mask_ok],
                    yerr=obs_fluxes_err[mask_ok],
                    ls="",
                    marker="o",
                    label="Observed fluxes",
                    markerfacecolor="None",
                    markersize=5,
                    markeredgecolor="xkcd:pastel purple",
                    color="xkcd:light indigo",
                    capsize=0.0,
                    zorder=3,
                    lw=1,
                )
            mask_uplim = np.logical_and(
                    np.logical_and(obs_fluxes > 0.0, obs_fluxes_err < 0.0),
                    obs_fluxes_err > -9990.0 * k_corr_SED,
                )
            if not mask_uplim.any() == False:
                ax1.errorbar(
                        filters_wl[mask_uplim],
                        obs_fluxes[mask_uplim],
                        yerr=obs_fluxes_err[mask_uplim],
                        ls="",
                        marker="v",
                        label="Observed upper limits",
                        markerfacecolor="None",
                        markersize=6,
                        markeredgecolor="g",
                        capsize=0.0,
                    )
            mask_noerr = np.logical_and( obs_fluxes > 0.0, obs_fluxes_err < -9990.0 * k_corr_SED)
            if not mask_noerr.any() == False:
                ax1.errorbar(
                        filters_wl[mask_noerr],
                        obs_fluxes[mask_noerr],
                        ls="",
                        marker="p",
                        markerfacecolor="None",
                        markersize=5,
                        markeredgecolor="r",
                        label="Observed fluxes, no errors",
                        capsize=0.0,
                    )

            # Work with residuals
            mask = np.where(obs_fluxes > 0.0)
            ax2.errorbar(
                    filters_wl[mask],
                    (obs_fluxes[mask] - mod_fluxes[mask]) / obs_fluxes[mask],
                    yerr=obs_fluxes_err[mask] / obs_fluxes[mask],
                    marker="_",
                    label="(Obs-Mod)/Obs",
                    color="k",
                    capsize=0.0,
                    ls="None",
                    lw=1,
                )
            ax2.plot([xmin, xmax], [0.0, 0.0], ls="--", color="k")
            ax2.set_xscale("log")
            ax2.minorticks_on()

            ax1.tick_params(
                    direction="in",
                    axis="both",
                    which="both",
                    top=True,
                    left=True,
                    right=True,
                    bottom=False,
                )
            ax2.tick_params(direction="in", axis="both", which="both", right=True)

            figure.subplots_adjust(hspace=0.0, wspace=0.0)

            ax1.set_xlim(xmin, xmax)

            if yrange[0] is not False:
                ymin = yrange[0]
            else:
                if not mask_uplim.any() == False:
                    ymin = min(
                            min(
                                np.min(obs_fluxes[mask_ok]),
                                np.min(obs_fluxes[mask_uplim]),
                            ),
                            min(
                                np.min(mod_fluxes[mask_ok]),
                                np.min(mod_fluxes[mask_uplim]),
                            ),
                        )
                elif not mask_ok.any() == False:
                    ymin = min(
                            np.min(obs_fluxes[mask_ok]),
                            np.min(mod_fluxes[mask_ok]),
                        )
                else:  # No valid flux (e.g., fitting only properties)
                    ymin = ax1.get_ylim()[0]
                ymin *= 1e-2

            if yrange[1] is not False:
                ymax = yrange[1]
            else:
                if not mask_uplim.any() == False:
                    ymax = max(
                            max(
                                np.max(obs_fluxes[mask_ok]),
                                np.max(obs_fluxes[mask_uplim]),
                            ),
                            max(
                                np.max(mod_fluxes[mask_ok]),
                                np.max(mod_fluxes[mask_uplim]),
                            ),
                        )
                elif not mask_ok.any() == False:
                    ymax = max(
                            np.max(obs_fluxes[mask_ok]),
                            np.max(mod_fluxes[mask_ok]),
                        )
                else:  # No valid flux (e.g., fitting only properties)
                    ymax = ax1.get_ylim()[1]
                ymax *= 1e2

            xmin = xmin if xmin < xmax else xmax - 1e1
            ymin = ymin if ymin < ymax else ymax - 1e1

            ax1.set_ylim(ymin, ymax)
            ax2.set_xlim(xmin, xmax)
            ax2.set_ylim(-1.0, 1.0)
            
            if sed_type == "lum":
                ax2.set_xlabel(r"Rest-frame wavelength [$\mu$m]")
                ax1.set_ylabel("Luminosity [W/nm]")
            else:
                ax2.set_xlabel(r"Observed $\lambda$ ($\mu$m)")
                ax1.set_ylabel(r"S$_\nu$ (mJy)")
                
            ax2.set_ylabel("Relative\nresidual")
            ax1.legend(fontsize=6, loc="best", frameon=False)
            ax2.legend(fontsize=6, loc="best", frameon=False)
            plt.setp(ax1.get_xticklabels(), visible=False)
            plt.setp(ax1.get_yticklabels()[1], visible=False)
            figure.suptitle(
                    f"Best model for {obs['id'].data[0]}\n (z={z:.3}, "
                    f"reduced χ²={mod['best.reduced_chi_square'].data[0]:.2})"
                )

            ax1.grid()
            ax2.grid()
        plt.show()
        #plt.close(figure)
      


In [ ]:
def get_SSPfrompcigale(t_best_model, mod,obs, filters , series=series,sed_type = "mJy", xrange = [10.*1e-3,5e4*1e-3],yrange = [False,False]):
    """

    parameters :
       t_best_model : Table of best fitted model (wrt wavelength)
       mod : Table of all fit result (line selected for the current spectrum)
       obs : Table of observed flux (in mJy)
       filters: dictionnary of filtername and pivot wavelength
       series : what to plot
       sed_type : mJy  Fnu in obs frame, lum : Llambda in restframe
       xrange : wavelength ranage in microns xrange = [100.*1e-3,1e4*1e-3]
    
    Need to convert flux density in mJy (mW/m2/Hz) into Luminosity W/nm and vice versa

    Fnu = 1/(4piDL^2) * lambda^2/c * L_lambda
    L_lambda = (4piDL^2) * (c/lambda^2) * F_nu

    No K correction is calculated
    
    """

    all_data = BRACKETOF A DICT

    if True:
        # get the table containning all seds (wrt wavelength)
        sed = t_best_model
        # the best model
        # wavelengths from nm to microns 
        wavelength_spec = sed["wavelength"].data * 1e-3
        
        # pivot wavelengths in filter from nm to microns
        filters_wl = (np.array([filt_wl for filt_wl in filters.values()]) * 1e-3)

        
        # retrieve observed fluxes (in mJy)
        obs_fluxes = np.array([obs[filt].data[0] for filt in filters.keys()])
        obs_fluxes_err = np.array([obs[filt + "_err"].data[0] for filt in filters.keys()])
        
        # retrieve the model fluxes (in mJy)
        mod_fluxes = np.array(
            [
                mod["best." + filt].data[0]
                if "best." + filt in mod.colnames
                else np.nan
                for filt in filters.keys()
            ])

        # retrieve the redshift from the observation
        if obs["redshift"].data[0] >= 0:
            z = float(obs["redshift"].data[0])
        else:  # Redshift mode
            z = float(mod["best.universe.redshift"].data[0])

        # the redshift factor   
        zp1 = 1.0 + z

        # surface of the sphere centered on the object of radius distance-luminosity
        surf = 4.0 * np.pi * mod["best.universe.luminosity_distance"].data[0] ** 2

        # define the range if wavelength (in microns)
        xmin = 0.9 * np.min(filters_wl) if xrange[0] is False else xrange[0]
        xmax = 1.1 * np.max(filters_wl) if xrange[1] is False else xrange[1]

        if sed_type == "lum":
            # L_lambda = (4piDL^2) * (c/lambda^2) * F_nu
            # 1 mJy = 10^-29 W/m2/Hz
            # Every values converted to SI 
            # Convert wavelength from microns to meters
            # need to convert lumi from W/m to W/nm 
            k_corr_SED = (1e-29 * surf * ctes.c.value/ (filters_wl * 1e-6)**2)*1e-9

            #print("k_corr_SED --> W/m2 = ",k_corr_SED)
            
            # convert fluxes from mJy to W/nm
            obs_fluxes *= k_corr_SED
            obs_fluxes_err *= k_corr_SED
            mod_fluxes *= k_corr_SED

            #for cname in sed.colnames[1:]:

            # I think this should not be applied (why 1e3)
            for cname in sed.colnames[2:]:
                #sed[cname] *= wavelength_spec * 1e3
                sed[cname] *=  1e0
                
            # set source rest frame
            filters_wl /= zp1
            wavelength_spec /= zp1
            xmin /= zp1
            xmax /= zp1
            
        elif sed_type == "mJy":
            k_corr_SED = 1.0
            # 1 W/m2/Hz = 10^29 mJy
            #Fnu = 1/(4piDL^2) * lambda^2/c * L_lambda
            fact = 1e29 * 1e-3 * wavelength_spec ** 2 / ctes.c.value / surf

            #print("fact --> mJy = ",fact)
            
            #for cname in sed.colnames[1:]:
            for cname in sed.colnames[2:]:
                sed[cname] *= fact
        else:
            print("ERROR Unknown plot type.")

        wsed = np.where((wavelength_spec > xmin) & (wavelength_spec < xmax))
       
        all_data.append(wavelength_spec)
        
        if (sed.columns[0][wsed].data > 0.0).any():    
            
           

            # Stellar emission
            if ( "stellar_attenuated" in series and "stellar.young" in sed.columns):
                spectrum =  sed["stellar.young"].data[wsed]  + sed["stellar.old"].data[wsed]

                if "nebular.absorption_young" in sed.columns:
                    spectrum += sed["nebular.absorption_young"].data[wsed]
                    spectrum += sed["nebular.absorption_old"].data[wsed]

                if "attenuation.stellar.young" in sed.columns:
                    spectrum += sed["attenuation.stellar.young"].data[wsed]
                    spectrum += sed["attenuation.stellar.old"].data[wsed]

                ax1.loglog(
                        wavelength_spec[wsed],
                        spectrum,
                        label="Stellar attenuated",
                        color="gold",
                        marker=None,
                        nonpositive="clip",
                        linestyle="-",
                        linewidth=1.0,
                    )
                #print("stellar_attenuated spectrum \t",spectrum)

            if ("stellar_unattenuated" in series and "stellar.young" in sed.columns):

                spectrum = sed["stellar.old"].data[wsed] + sed["stellar.young"].data[wsed]
                
                ax1.loglog(
                        wavelength_spec[wsed],
                        spectrum,
                        label="Stellar unattenuated",
                        color="xkcd:deep sky blue",
                        marker=None,
                        nonpositive="clip",
                        linestyle="--",
                        linewidth=1.0,
                    )
                #print("stellar_unattenuated spectrum \t",spectrum)

            # Nebular emission
            if "nebular" in series and "nebular.lines_young" in sed.columns:
                spectrum = (
                        sed["nebular.lines_young"].data[wsed]
                        + sed["nebular.lines_old"].data[wsed]
                        + sed["nebular.continuum_young"].data[wsed]
                        + sed["nebular.continuum_old"].data[wsed]
                    )

                if "attenuation.nebular.lines_young" in sed.columns:
                        spectrum += sed["attenuation.nebular.lines_young"].data[wsed]
                        spectrum += sed["attenuation.nebular.lines_old"].data[wsed]
                        spectrum += sed["attenuation.nebular.continuum_young"].data[wsed]
                        spectrum += sed["attenuation.nebular.continuum_old"].data[wsed]

                ax1.loglog(
                        wavelength_spec[wsed],
                        spectrum,
                        label="Nebular emission",
                        color="xkcd:true green",
                        marker=None,
                        nonpositive="clip",
                        linewidth=1.0,
                    )
                #print("nebular  spectrum \t",spectrum)

            # Dust emission Draine & Li
            if "dust" in series and "dust.Umin_Umin" in sed.columns:
                spectrum =  sed["dust.Umin_Umin"].data[wsed] + sed["dust.Umin_Umax"].data[wsed]
                ax1.loglog(
                        wavelength_spec[wsed],spectrum,
                        label="Dust emission",
                        color="xkcd:bright red",
                        marker=None,
                        nonpositive="clip",
                        linestyle="-",
                        linewidth=1.0,
                    )
                #print("Dust Draine Li  spectrum \t",spectrum)

            # Dust emission Dale
            if "dust" in series and "dust" in sed.columns:
                spectrum = sed["dust"].data[wsed]
                ax1.loglog(
                        wavelength_spec[wsed],
                        spectrum,
                        label="Dust emission",
                        color="xkcd:bright red",
                        marker=None,
                        nonpositive="clip",
                        linestyle="-",
                        linewidth=1.0,
                    )
                #print("Dust spectrum \t",spectrum)

            # AGN emission
            if "agn" in series and ( "agn.fritz2006_torus" in sed.columns or "agn.SKIRTOR2016_torus" in sed.columns):
                if "agn.fritz2006_torus" in sed.columns:
                    agn_sed = (
                            sed["agn.fritz2006_polar_dust"]
                            + sed["agn.fritz2006_torus"]
                            + sed["agn.fritz2006_disk"]
                        )
                elif "agn.SKIRTOR2016_torus" in sed.columns:
                    agn_sed = (
                            sed["agn.SKIRTOR2016_polar_dust"]
                            + sed["agn.SKIRTOR2016_torus"]
                            + sed["agn.SKIRTOR2016_disk"]
                        )
                if "xray.agn" in sed.columns:
                    agn_sed += sed["xray.agn"]
                if "radio.agn" in sed.columns:
                    agn_sed += sed["radio.agn"]
                    
                ax1.loglog(
                        wavelength_spec[wsed],
                        agn_sed[wsed],
                        label="AGN emission",
                        color="xkcd:apricot",
                        marker=None,
                        nonpositive="clip",
                        linestyle="-",
                        linewidth=1.0,
                    )

            # Radio emission
            if "radio" in series and "radio.sf_nonthermal" in sed.columns:
                ax1.loglog(
                        wavelength_spec[wsed],
                        sed["radio.sf_nonthermal"][wsed],
                        label="Radio SF nonthermal",
                        color="brown",
                        marker=None,
                        nonpositive="clip",
                        linestyle="-",
                        linewidth=1.0,
                    )

            if "model" in series:
                ax1.loglog(
                        wavelength_spec[wsed],
                        sed["L_lambda_total"].data[wsed],
                        label="Model spectrum",
                        color="k",
                        nonpositive="clip",
                        linestyle="-",
                        linewidth=1.5,
                    )
                #print("model", sed["L_lambda_total"].data[wsed])

            
            ax1.set_autoscale_on(False)
            
            s = np.argsort(filters_wl)
            filters_wl = filters_wl[s]
            mod_fluxes = mod_fluxes[s]
            obs_fluxes = obs_fluxes[s]
            obs_fluxes_err = obs_fluxes_err[s]
            ax1.scatter(
                    filters_wl,
                    mod_fluxes,
                    marker="o",
                    color="xkcd:strawberry",
                    s=8,
                    zorder=3,
                    label="Model fluxes",
                )
            mask_ok = np.logical_and(obs_fluxes > 0.0, obs_fluxes_err > 0.0)
            ax1.errorbar(
                    filters_wl[mask_ok],
                    obs_fluxes[mask_ok],
                    yerr=obs_fluxes_err[mask_ok],
                    ls="",
                    marker="o",
                    label="Observed fluxes",
                    markerfacecolor="None",
                    markersize=5,
                    markeredgecolor="xkcd:pastel purple",
                    color="xkcd:light indigo",
                    capsize=0.0,
                    zorder=3,
                    lw=1,
                )
            mask_uplim = np.logical_and(
                    np.logical_and(obs_fluxes > 0.0, obs_fluxes_err < 0.0),
                    obs_fluxes_err > -9990.0 * k_corr_SED,
                )
            if not mask_uplim.any() == False:
                ax1.errorbar(
                        filters_wl[mask_uplim],
                        obs_fluxes[mask_uplim],
                        yerr=obs_fluxes_err[mask_uplim],
                        ls="",
                        marker="v",
                        label="Observed upper limits",
                        markerfacecolor="None",
                        markersize=6,
                        markeredgecolor="g",
                        capsize=0.0,
                    )
            mask_noerr = np.logical_and( obs_fluxes > 0.0, obs_fluxes_err < -9990.0 * k_corr_SED)
            if not mask_noerr.any() == False:
                ax1.errorbar(
                        filters_wl[mask_noerr],
                        obs_fluxes[mask_noerr],
                        ls="",
                        marker="p",
                        markerfacecolor="None",
                        markersize=5,
                        markeredgecolor="r",
                        label="Observed fluxes, no errors",
                        capsize=0.0,
                    )

            # Work with residuals
            mask = np.where(obs_fluxes > 0.0)
            ax2.errorbar(
                    filters_wl[mask],
                    (obs_fluxes[mask] - mod_fluxes[mask]) / obs_fluxes[mask],
                    yerr=obs_fluxes_err[mask] / obs_fluxes[mask],
                    marker="_",
                    label="(Obs-Mod)/Obs",
                    color="k",
                    capsize=0.0,
                    ls="None",
                    lw=1,
                )
            ax2.plot([xmin, xmax], [0.0, 0.0], ls="--", color="k")
            ax2.set_xscale("log")
            ax2.minorticks_on()

            ax1.tick_params(
                    direction="in",
                    axis="both",
                    which="both",
                    top=True,
                    left=True,
                    right=True,
                    bottom=False,
                )
            ax2.tick_params(direction="in", axis="both", which="both", right=True)

            figure.subplots_adjust(hspace=0.0, wspace=0.0)

            ax1.set_xlim(xmin, xmax)

            if yrange[0] is not False:
                ymin = yrange[0]
            else:
                if not mask_uplim.any() == False:
                    ymin = min(
                            min(
                                np.min(obs_fluxes[mask_ok]),
                                np.min(obs_fluxes[mask_uplim]),
                            ),
                            min(
                                np.min(mod_fluxes[mask_ok]),
                                np.min(mod_fluxes[mask_uplim]),
                            ),
                        )
                elif not mask_ok.any() == False:
                    ymin = min(
                            np.min(obs_fluxes[mask_ok]),
                            np.min(mod_fluxes[mask_ok]),
                        )
                else:  # No valid flux (e.g., fitting only properties)
                    ymin = ax1.get_ylim()[0]
                ymin *= 1e-2

            if yrange[1] is not False:
                ymax = yrange[1]
            else:
                if not mask_uplim.any() == False:
                    ymax = max(
                            max(
                                np.max(obs_fluxes[mask_ok]),
                                np.max(obs_fluxes[mask_uplim]),
                            ),
                            max(
                                np.max(mod_fluxes[mask_ok]),
                                np.max(mod_fluxes[mask_uplim]),
                            ),
                        )
                elif not mask_ok.any() == False:
                    ymax = max(
                            np.max(obs_fluxes[mask_ok]),
                            np.max(mod_fluxes[mask_ok]),
                        )
                else:  # No valid flux (e.g., fitting only properties)
                    ymax = ax1.get_ylim()[1]
                ymax *= 1e2

            xmin = xmin if xmin < xmax else xmax - 1e1
            ymin = ymin if ymin < ymax else ymax - 1e1

            ax1.set_ylim(ymin, ymax)
            ax2.set_xlim(xmin, xmax)
            ax2.set_ylim(-1.0, 1.0)
            
            if sed_type == "lum":
                ax2.set_xlabel(r"Rest-frame wavelength [$\mu$m]")
                ax1.set_ylabel("Luminosity [W/nm]")
            else:
                ax2.set_xlabel(r"Observed $\lambda$ ($\mu$m)")
                ax1.set_ylabel(r"S$_\nu$ (mJy)")
                
            ax2.set_ylabel("Relative\nresidual")
            ax1.legend(fontsize=6, loc="best", frameon=False)
            ax2.legend(fontsize=6, loc="best", frameon=False)
            plt.setp(ax1.get_xticklabels(), visible=False)
            plt.setp(ax1.get_yticklabels()[1], visible=False)
            figure.suptitle(
                    f"Best model for {obs['id'].data[0]}\n (z={z:.3}, "
                    f"reduced χ²={mod['best.reduced_chi_square'].data[0]:.2})"
                )

            ax1.grid()
            ax2.grid()
        plt.show()
        #plt.close(figure)
      


## Table must be copied otherwise would be overwritten

In [ ]:
plots_frompcigale(t_best_model= t_best.copy(), mod = t_res_sel.copy(), obs= FORS2_sel.copy(), filters = dict_filters, series=series, 
                  sed_type = "mJy")

## Loops

In [ ]:
list_of_keys = np.array(list(spec_dict.keys())) 

In [ ]:
all_nums = []
for key in list_of_keys:
    num_str = re.findall("^SPEC(.+)",key)[0]
    all_nums.append(int(num_str)) 
all_nums = np.array(all_nums)

In [ ]:
sorted_indexes = np.argsort(all_nums)

In [ ]:
list_of_keys_sorted = list_of_keys[np.argsort(all_nums)]

In [ ]:
for spec_name in list_of_keys_sorted:

    # retrieve the filename
    file_best_model = os.path.join(path,spec_dict[spec_name]['best_model'])
    file_sfh = os.path.join(path,spec_dict[spec_name]['SFH'])

    # readthe table
    t_best = Table.read(file_best_model)
    t_sfh = Table.read(file_sfh)

    #get the result for that spectrum, t_res table table of all results outside this loop    
    cut_select = t_res["id"] == spec_name
    t_res_sel = t_res[cut_select]

    # data and select this spectrum row
    cut_fors2select = FORS2["id"] == spec_name
    FORS2_sel = FORS2[cut_fors2select]


    plots_frompcigale(t_best_model= t_best.copy(), mod = t_res_sel.copy(), obs= FORS2_sel.copy(), filters = dict_filters, series=series, 
                  sed_type = "mJy")
    